In [ ]:
import os
!git clone https://github.com/aldo-fsm/fake-news-detection.git
os.chdir('fake-news-detection')
# !pip install pipfile-requirements
# !pipfile2req > requirements.txt
# !pip install -r requirements.txt
!pip install ktrain
! mkdir models #&& mv ../SVM_rbf_20000feat_acc_0.909741 ./models/

In [ ]:
!cp -r '../drive/MyDrive/TCC/Detecção de Fake News/Models/' './models/'
!mv models/Models/* models/

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import display
import plotly_express as px
import plotly.io as pio
pio.renderers.default = 'colab'

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from preprocessing import stopwords, preprocess, putEndDot
from datasets import fake_br_corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
dataset = fake_br_corpus.loadTest()
dataset['text'] = dataset.text.apply(putEndDot)

In [ ]:
import joblib
model = joblib.load('models/SVM_rbf_20000feat_acc_0.909741')

In [ ]:
from preprocessing.sentence_ranking_transformer import SentenceRankingTransformer
ranker = SentenceRankingTransformer(model, 'fake')

In [ ]:
dataset['rankedText'] = ranker.fit_transform(dataset.text)

In [ ]:
dataset[['isFake', 'isTrue']] = dataset.label.apply(lambda label: pd.Series([1,0]) if label == 'fake' else pd.Series([0,1]))

In [ ]:
import ktrain
from ktrain import text

In [ ]:
from utils import evaluation
from sklearn.metrics import accuracy_score

In [ ]:
ytrue = dataset.label.apply(lambda label: 'isFake' if label == 'fake' else 'isTrue').values

In [ ]:
def evaluateAll(Xtest, ytrue, paths):
  results = []
  for path in predictorPaths:
    print(path)
    predictor = ktrain.load_predictor(path)
    ypred = evaluation.evaluate(predictor, Xtest, ytrue)
    results.append(dict(
        model=path,
        accuracy=accuracy_score(ytrue, ypred)
    ))
    print('\n')
  return results

In [ ]:
results = []

In [ ]:
predictorPaths = [
  'models/bert-ktrain/1_epoch_val_accuracy_0.9525/predictor/',
  'models/bert-ktrain/6_epoch_val_accuracy_0.9754/predictor/',
  'models/invalid/biased_ktrain-bert/1_epoch_val_accuracy_0.9623/predictor/',
  'models/invalid/biased_ktrain-bert/6_epoch_val_accuracy_0.9836/predictor/',
]
Xtest = dataset.rankedText.values
results.extend(evaluateAll(
  Xtest,4
  ytrue,
  predictorPaths
))

models/bert-ktrain/1_epoch_val_accuracy_0.9525/predictor/
              precision    recall  f1-score   support

      isFake       0.89      0.94      0.91       553
      isTrue       0.94      0.87      0.90       527

    accuracy                           0.91      1080
   macro avg       0.91      0.91      0.91      1080
weighted avg       0.91      0.91      0.91      1080

accuracy: 0.9092592592592592


models/bert-ktrain/6_epoch_val_accuracy_0.9754/predictor/
              precision    recall  f1-score   support

      isFake       0.89      0.96      0.92       553
      isTrue       0.95      0.87      0.91       527

    accuracy                           0.91      1080
   macro avg       0.92      0.91      0.91      1080
weighted avg       0.92      0.91      0.91      1080

accuracy: 0.9148148148148149


models/invalid/biased_ktrain-bert/1_epoch_val_accuracy_0.9623/predictor/
              precision    recall  f1-score   support

      isFake       0.83      0.99      0

In [ ]:
predictorPaths = [
  'models/bert-ktrain-non-ranked/1_epoch_val_accuracy_0.9885/predictor/'
]
Xtest = dataset.text.values
results.extend(evaluateAll(
  Xtest,
  ytrue,
  predictorPaths
))

models/bert-ktrain-non-ranked/1_epoch_val_accuracy_0.9885/predictor/
              precision    recall  f1-score   support

      isFake       0.96      0.99      0.97       553
      isTrue       0.99      0.95      0.97       527

    accuracy                           0.97      1080
   macro avg       0.97      0.97      0.97      1080
weighted avg       0.97      0.97      0.97      1080

accuracy: 0.9712962962962963




In [ ]:
resultsDf = pd.DataFrame(results)
resultsDf

,model,accuracy
0,models/bert-ktrain-non-ranked/1_epoch_val_accu...,0.971296


In [ ]:
from datetime import datetime

In [ ]:
date = datetime.now().isoformat().replace(':','_')
resultsDf.to_csv(f'bert-results_{date}.csv')